# Despliegue de Resultados : Dash

Es esta clase se interactúa con notebooks y aplicaciones locales para el despliegue de modelos de analítica.

In [17]:
# Importamos paquetes necesarios
import pandas as pd 
import numpy as np
import plotly.graph_objs as go
from apyori import apriori

In [2]:
panaderia = pd.read_csv('BreadBasket_DMS.csv') # Carga y exploración de datos

In [3]:
panaderia.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [4]:
panaderia['Item'].value_counts()

Coffee                           5471
Bread                            3325
Tea                              1435
Cake                             1025
Pastry                            856
NONE                              786
Sandwich                          771
Medialuna                         616
Hot chocolate                     590
Cookies                           540
Brownie                           379
Farm House                        374
Muffin                            370
Alfajores                         369
Juice                             369
Soup                              342
Scone                             327
Toast                             318
Scandinavian                      277
Truffles                          193
Coke                              185
Spanish Brunch                    172
Fudge                             159
Baguette                          152
Jam                               149
Tiffin                            146
Mineral wate

# Reglas de Asociación

¿Podemos encontrar bienes que sean comprados juntos con mucha frecuencia?

<center>
<img src='im/ar_image.png'> 
</center>

- Organizar mejor estanterías de almacenes para mejorar sus ingresos.
- Hacer mercadeo cruzado (up/cross-selling) de ciertos productos relacionados.
- Segmentar clientes de acuerdo con sus patrones de consumo.
- Diseñar mejores experiencias para usuarios en páginas web.

Una **regla** se define como $A\to B$, con $A, B$ canastas  sin elementos en comun.

$A$ se conoce como el _antecedente_ de la regla, y $B$ el _consecuente_.

**Soporte:** Popularidad de una canasta.

El soporte de una canasta $A$, $supp(A)$ es la proporción de las transacciones en la base de datos en las que aparece la canasta $A$:
$$supp(A) = \frac{\text{No. de transacciones en las que aparece }A}{\text{No. total de transacciones}}$$

**Confianza**

La confianza mide la probabilidad de que el consecuente sea adquirido dado que ya se tiene el antecedente: $P(B|A)$.

La confianza de una regla $A\to B$ se define:
$$conf(A\to B) = \frac{supp(A\cup B)}{supp(A)}$$

**Lift**

$$lift(A\to B) = \frac{conf(A\to B)}{supp(B)} = \frac{supp(A\cup B)}{supp(A)*supp(B)}$$

Mide la probabilidad de que se compre la canasta $B$ dado que ya se tiene la canasta $A$ ($P(B|A)$), teniendo en cuenta la popularidad de la canasta $B$.

Un lift mayor (menor) que $1$ indica que la canasta $B$ incrementa (reduce) su probabilidad de ser adquirida si ya se tiene la cansata $A$

In [5]:
# El algoritmo apriori exige que la estructura de datos que recibe consista de una lista de listas, en la que cada lista interna contenga los items de la transacción.
# Hay varias formas válidas de hacer esto, a continuación una de estas.
data = [list(panaderia.loc[panaderia['Transaction'] == i, 'Item']) for i in panaderia['Transaction'].unique()] 

In [6]:
len(data)

9531

In [7]:
a_rules = apriori(data, min_support = 0.005, min_confidence = 0.20) # Ejecutamos el algoritmo apriori con un soporte y una confianza mínimos.

In [8]:
a_results = list(a_rules) # Transformamos el resultado en lista

In [9]:
len(a_results) #¿Cuántas reglas identificó el algoritmo?

61

In [10]:
a_results[3] # Exploramos una regla para estudiar su estructura

RelationRecord(items=frozenset({'Alfajores', 'Coffee'}), support=0.019515265974189486, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Alfajores'}), items_add=frozenset({'Coffee'}), confidence=0.5406976744186047, lift=1.1381160633577123)])

In [18]:
soportes = [regla[1] for regla in a_results] # Extraemos los soportes de las reglas

In [19]:
confianzas = [regla[2][0][2] for regla in a_results] # Extraemos las confianzas de las reglas.

In [21]:
lifts = [regla[2][0][3] for regla in a_results] # Extraemos los lifts de las reglas.

In [23]:
antecedentes = [', '.join(list(regla[2][0][0])) for regla in a_results] # Extraemos los antecedentes de las reglas.

In [24]:
consecuentes = [', '.join(list(regla[2][0][1])) for regla in a_results] # Extraemos los consecuentes de las reglas.

In [42]:
# Resumimos la extracción anterior en una función, que en adición da formato a los valores extraídos.
def extraer_info(regla):
    
    antecedente = ', '.join(regla[2][0][0])
    consecuente = ', '.join(regla[2][0][1])
    soporte = regla[1]
    confianza = regla[2][0][2]
    lift = regla[2][0][3]
    
    return '{:s} --> {:s} <br><br>Soporte: {:.3f} <br>Confianza: {:.3f} <br>Lift: {:.3f}'.format(antecedente, consecuente, soporte, confianza, lift)
    

In [43]:
infos = [extraer_info(regla) for regla in a_results] #Extraemos la información completa de las reglas con la función construida

In [32]:
n_antecesores = [len(list(regla[2][0][0])) for regla in a_results] # Extraemos el número de antecesores de cada regla

In [44]:
# Utilizamos plotly para graficar interactivamente las reglas de asociación resultantes.
# El eje x coresponde al soporte, y el eje y a la confianza de cada regla. 
# El tamaño del gráfico de disporsión corresponde al lift, y el color al número de antecesores. 
# Finalmente asignamos el texto de la información extraída a cada punto del gráfico
fig = go.Figure(data = [go.Scatter(x = soportes, y = confianzas, mode='markers',
                                   marker = {'size': np.multiply(lifts, 10),
                                             'color': n_antecesores},
                                   text = infos, 
                                   hoverinfo='text')])
fig.show(renderer = 'browser')

### Parametrización de modelos

¿De qué variables/parámetros depende nuestro modelo?

In [34]:
# Creamos función que genera las reglas de asociación dado un soporte y una confianza mínima
def reglas_asociacion(min_soporte, min_confianza):
    a_rules = apriori(data, min_support = min_soporte, min_confidence = min_confianza)
    a_results = list(a_rules)
    
    return a_results

In [38]:
# La función input permite interactuar con el notebook para ingresar valores deseados por el usuario.
# El valor que ingrese queda guardado (como texto) en la variable especificada
soporte = float(input('Ingrese soporte mínimo: '))
confianza = float(input('Ingrese confianza mínima: '))

# Con el soporte y la confianza ingresada por el usuario generamos nuevas reglas de asociación con la función creada
reglas = reglas_asociacion(float(soporte), float(confianza))

Ingrese soporte mínimo:  0.002
Ingrese confianza mínima:  0.3


In [45]:
# Extraemos la información de las reglas resultantes 
soportes = [regla[1] for regla in reglas]
confianzas = [regla[2][0][2] for regla in reglas]
lifts = [regla[2][0][3] for regla in reglas]
infos = [extraer_info(regla) for regla in reglas]
n_antecesores = [len(list(regla[2][0][0])) for regla in reglas]

In [46]:
# Graficamos las nuevas reglas resultantes
fig = go.Figure(data = [go.Scatter(x = soportes, y = confianzas, mode='markers',
                                   marker = {'color': n_antecesores,
                                             'size': np.multiply(lifts, 10)},
                                   text = infos, 
                                   hoverinfo='text')])
fig.show(renderer = 'browser')

# Generación de Aplicaciones Locales con Dash

Dash es un _framework_ de Python para crear aplicaciones web. Ideal para crear aplicaciones de visualización de datos con interfaces de usuario altamente personalizadas en Python puro. Es especialmente adecuado para cualquier persona que trabaje con datos en Python. Las aplicaciones locales se acceden vía navegadores web. 

Más información: 
- [Página principal Dash](https://plot.ly/dash/) 
- [Tutorial Dash](https://dash.plot.ly/)

Para ejecutar una aplicación Dash debe guardar el código en formato _.py_ (ideal trabajar en Spyder) y luego correr el siguiente comando desde la Anaconda Prompt, ubicado en el directorio en el que se encuentra el archivo (esto se logra con el comando _cd_):

_python aplicacion.py_

Verá algo así:

<center>
<img src='im/ejec_dash.PNG'> 
</center>

Luego, simplemente entre desde su navegador de preferencia a http://127.0.0.1:8050